**Load Packages and Import Data**

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

df = pd.read_csv('carData.csv')

**Split Data**

In [3]:
X = df[df.columns[:-1]]
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [9]:
hp_data = df[df['engine'].str.contains('HP')]

hp_data['hp'] = hp_data['engine'].str.split().str[0].str.strip('HP')

hp_data = hp_data[['brand','model','model_year','hp','price']].sort_values(by=['hp','price'],ascending=False)
hp_data.to_csv('test.csv')

C:\Users\amans\AppData\Local\Temp\ipykernel_18500\1092339245.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hp_data['hp'] = hp_data['engine'].str.split().str[0].str.strip('HP')
